In [1]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()
# 初始化模型
model = CNNPointDetector()
model.load_state_dict(torch.load('fan_corner_detector_1114.pth'))  # 加载模型参数
model.train()

/opt/miniconda3/envs/tvdi/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
/var/folders/pn/9sxhhyf54lv07cdggqllxgg00000gn/T/ipykernel_97582/1811785209.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

CNNPointDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [2]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points_1114_2.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about %.2f mins left' % ((num_epochs-epoch)/11))

[ WARN:0@1382.910] global loadsave.cpp:241 findDecoder imread_('.\pic_3\128.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [13]:
torch.save(model.state_dict(), './fan_corner_detector_1114.pth')